In [1]:
import numpy as np

In [3]:
data = np.load("/home/droplab/workspace/lerobot_piper/my_scripts/action_chunks.npz")

In [5]:
data["action"]

array([[-3.9900765e-01,  1.8907247e+00, -1.8925357e+00, ...,
         8.1687653e-01,  1.7624459e-01,  9.2451926e-04],
       [-3.9847836e-01,  1.8803532e+00, -1.8828855e+00, ...,
         8.1762755e-01,  1.7651504e-01,  9.3531143e-04],
       [-3.9725763e-01,  1.8337820e+00, -1.8419601e+00, ...,
         8.1243050e-01,  1.7898813e-01,  9.4824051e-04],
       ...,
       [-3.7311730e-01,  5.8785295e-01, -3.7313771e-01, ...,
         8.7874758e-01, -3.7581652e-02,  4.4477560e-02],
       [-3.7298745e-01,  5.8610409e-01, -3.7351060e-01, ...,
         8.7946385e-01, -3.7304148e-02,  4.4409875e-02],
       [-3.7323117e-01,  5.8593869e-01, -3.7426701e-01, ...,
         8.7772727e-01, -3.5074532e-02,  4.3908954e-02]],
      shape=(100, 14), dtype=float32)